# japanese-gpt-neox-3.6b-instruction-sft を SageMaker で Hosting
このノートブックは、rinna の japanese-gpt-neox-3.6b-instruction-sft モデルを、ローカルで推論し、それを SageMaker Inference に移植するノートブックです。  
モデルの詳細については [Hugging Face apanese-gpt-neox-3.6b-instruction-sft](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft) を参照ください。 一度ローカルで推論する都合上、ml.g5.2xlarge インスタンスを使用します。
SageMaker Notebooks の conda_pytorch_p39 カーネルと、SageMaker Studio Notebook の PyTorch 1.13 Python 3.9 GPU Optimized カーネルで動いた実績があります。  
ノートブックは外部ファイルを参照していないので、どのディレクトリに配置してあっても動作します。  


また、ノートブックを動かすにあたって、各セルを実行すれば動きますが、どのように動くかなどについては、[AI/ML DarkPark](https://www.youtube.com/playlist?list=PLAOq15s3RbuL32mYUphPDoeWKUiEUhcug) の特に [Amazon SageMaker 推論 Part2すぐにプロダクション利用できる！モデルをデプロイして推論する方法 【ML-Dark-04】【AWS Black Belt】](https://youtu.be/sngNd79GpmE) をご参照ください。

## ローカル推論
SageMaker 推論エンドポイントにホスティングする前に、このNotebook上で動作確認を行う
### ローカルで動かすためのライブラリをインストール
必要なモジュールをインストールする

In [ ]:
pip install transformers einops SentencePiece

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

### モデルのダウンロード
tokenizer と model をダウンロードします。[How to use the model](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft#how-to-use-the-model) に沿って実行します。

In [ ]:
%%time 

tokenizer = AutoTokenizer.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-sft", use_fast=False
)

以下のセルはモデルを DL して読み込むため 3 分ほど時間がかかります。

In [ ]:
%%time

model = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt-neox-3.6b-instruction-sft", 
).to("cuda:0")

### モデルの保存
ローカルで推論する前に、モデルをストレージに出力して、再度読み込みます。  
SageMaker で Hosting する際はモデルをファイルから読み込むことが一般的で、ローカルで動かすときもその方法に則って行うと、SageMaker に移植しやすいためにこの手順を入れています。

In [ ]:
!rm -rf './model'
!mkdir -p './model/code'
model_dir = './model'
tokenizer.save_pretrained(model_dir)
model.save_pretrained(model_dir)

メモリを解放します(OOM 対策)

In [ ]:
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

### モデルの再ロード
ファイルから tokenizer と model をロードします。  
model は 7 分程度かかります。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)

In [ ]:
%%time

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
).to("cuda:0")

### 推論する
prompt の形式は [japanese-gpt-neox-3.6b-instruction-sft](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-sft#japanese-gpt-neox-3.6b-instruction-sft) にあるとおり、以下にすると良い結果を得られやすいです。
* プロンプトはユーザーとシステムの会話形式で与える
* 各発言は、以下形式に則る  
    `{ユーザー, システム} : {発言}`
* プロンプトの末尾は`システム:` で終了させる
* 改行は`<NL>`を利用し、発言はすべて `<NL>` で区切る必要がある

以下はプロンプトの例です。`<NL>`の埋め込みが大変なので、改行で書いて後で置換します。

In [ ]:
prompt = '''ユーザー: 世界自然遺産を列挙してください。
システム: 膨大な数です。例えば国で絞ってください。
ユーザー: イギリスでお願いします。
システム:'''.replace('\n','<NL>')
print(prompt)

In [ ]:
%%time
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

In [ ]:
%%time
with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.01,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
output = output.replace("<NL>", "\n")
print(output)

無事動いたらローカルでやっていたことを SageMaker の Hosting を使って再現します。

## SageMaker による推論

### モジュールのロードと定数の設定

In [ ]:
import sagemaker
import boto3
sm = boto3.client('sagemaker')
role = sagemaker.get_execution_role()

### 推論コードの作成
先程実行したコードをもとに記述していきます。  
まずは必要なモジュールを記述した requirements.txt を用意します。  
今回は [deep-learning-containers](https://github.com/aws/deep-learning-containers)の HuggingFace のコンテナを使います。  
einops と SentencePiece が不足しているので requirements.txt に記載します。

In [ ]:
%%writefile model/code/requirements.txt
einops
SentencePiece

先程実行したコードを SageMaker Inference 向けに改変します。
1. `model_fn` でモデルを読み込みます。先程は huggingface のモデルを直接ロードしましたが、`model_dir` に展開されたモデルを読み込みます。
2. `input_fn` で前処理を行います。
    * json 形式のみを受け付け他の形式は弾くようにします。
    * json 文字列を dict 形式に変換して `return` します。
3. `predict_fn` で推論します。`temperature` などのパラメータも合わせて入力します。
4. `output_fn` で json 形式にして `return` します。

In [ ]:
%%writefile model/code/inference.py
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import subprocess
from subprocess import PIPE

DEVICE = 'cuda:0'

def model_fn(model_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)
    model = AutoModelForCausalLM.from_pretrained(model_dir).to(DEVICE)
    return {'tokenizer':tokenizer,'model':model}

def input_fn(data, content_type):
    if content_type == 'application/json':
        data = json.loads(data)
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

def predict_fn(data, model):
    prompt = data['prompt']
    token_ids = model['tokenizer'].encode(prompt, add_special_tokens=False, return_tensors="pt")
    do_sample = data['do_sample']
    max_new_tokens = data['max_new_tokens']
    temperature = data['temperature']
    
    with torch.no_grad():
        output_ids = model['model'].generate(
            token_ids.to(DEVICE),
            do_sample=True,
            max_new_tokens=128,
            temperature=0.01,
            pad_token_id=model['tokenizer'].pad_token_id,
            bos_token_id=model['tokenizer'].bos_token_id,
            eos_token_id=model['tokenizer'].eos_token_id
        )
    output = model['tokenizer'].decode(output_ids.tolist()[0][token_ids.size(1):])
    output = output.replace("<NL>", "\n")
    
    return output

def output_fn(data, accept_type):
    if accept_type == 'application/json':
        data = json.dumps({'result' : data})
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

### モデルアーティファクトの作成と S3 アップロード
アーティファクト(推論コード + モデル)を tar.gz に固めます。時間がかかるので `pigz` で並列処理を行います。  
ml.g5.2xlarge で 9 分ほどかかります。

※ SageMaker Studio を使っている場合は pigz が入っていないので、以下セルのコメントを解除してインストールしてください。

In [ ]:
# !apt update -y
# !apt install pigz -y

In [ ]:
%%time

!rm model.tar.gz
%cd model/
!tar  cv ./ | pigz -p 8 > ../model.tar.gz # 8 並列でアーカイブ
%cd ..

In [ ]:
%%time

model_s3_uri = sagemaker.session.Session().upload_data(
    'model.tar.gz',
    key_prefix='japanese-gpt-neox-3.6b-instruction-sft'
)
print(model_s3_uri)

### SageMaker SDK を用いてデプロイ
使用している API の詳細は以下を確認してください。  
[Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/index.html)

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
region = boto3.session.Session().region_name

In [ ]:
# 名前の設定
model_name = 'japanese-gpt-neox-3-6b-instruction-sft'
endpoint_config_name = model_name + 'Config'
endpoint_name = model_name + 'Endpoint'

In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework='huggingface',
    region=region,
    version='4.26',
    image_scope='inference',
    base_framework_version='pytorch1.13',
    instance_type = 'ml.g5.xlarge'
)

In [ ]:
huggingface_model = HuggingFaceModel(
    model_data = model_s3_uri,
    role = role,
    image_uri = image_uri
)

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

### SageMaker SDK で推論
model_fn の実行に時間がかかってしまい、エンドポイントが IN_SERVICE になっても、初回推論はしばらく動かないことがあります。  
CloudWatch Logs に以下のような表示がある場合はしばらく待てば使えるようになります。  
`[WARN] pool-3-thread-1 com.amazonaws.ml.mms.metrics.MetricCollector - worker pid is not available yet.`  
だいたい 6 分くらいかかるため、リトライを入れています。

In [ ]:
# prompt 確認
print(prompt)

In [ ]:
from time import sleep
request = {
    'prompt' : prompt,
    'max_new_tokens' : 128,
    'do_sample' : True,
    'temperature' : 0.01,
}

for i in range(10):
    try:
        output = predictor.predict(request)['result']
        break
    except:
        sleep(60)

In [ ]:
print(output)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## boto3 でデプロイと推論
標準だと SageMaker SDK が入っていない環境からデプロイや推論する場合(例:AWS Lambda など)は、boto3 でデプロイや推論することも多いです。  
以下のセルは boto3 で実行する方法を記述しています。
各 API の詳細は Document を確認してください。  
[SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html)  
[SageMakerRuntime](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html)  

In [ ]:
import json
sm = boto3.client('sagemaker')
smr = boto3.client('sagemaker-runtime')
endpoint_inservice_waiter = sm.get_waiter('endpoint_in_service')

モデルの作成

In [ ]:
response = sm.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image_uri,
        'ModelDataUrl': model_s3_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
            'SAGEMAKER_REGION': region,
        }
    },
    ExecutionRoleArn=role,
)

エンドポイントコンフィグの作成

In [ ]:
response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g5.2xlarge',
        },
    ]
)

エンドポイントの作成

In [ ]:
response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

エンドポイント作成の完了を待つ

In [ ]:
endpoint_inservice_waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={'Delay': 5,}
)

推論を行います。  
ただし初回推論時のみモデルのロードに 7 分ほどかかるため、先程同様リトライを入れています。

In [ ]:
# prompt 確認
print(request)

In [ ]:
%%time

# 推論
smr = boto3.client('sagemaker-runtime')

for i in range(10):
    try:
        response = smr.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Accept='application/json',
            Body=json.dumps(request)
        )
        break
    except:
        sleep(60)
output = json.loads(response['Body'].read().decode('utf-8'))['result']
print(output)

お片付け

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_model(ModelName=model_name)